In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from model import create_model
from data import data_generator, complete_record_generator

In [ ]:
dataset_dir = ''

valid_generator = data_generator(complete_record_generator(
    dataset_dir=dataset_dir,
    set_name='valid',
    batch_size=1
))

In [ ]:
model = create_model()
model.load_weights('')

In [ ]:
plt.figure(figsize=(16, 12))

for i in range(0, 9):
    inputs, outputs = next(valid_generator)
    preds = model.predict(inputs)

    f0_pred = np.argmax(preds[0], axis=-1)
    f0_gt = np.argmax(outputs['f0_categorical'][0], axis=-1)

    plt.subplot(3, 3, i + 1)
    plt.plot(range(1000), f0_pred, label='Pred')
    plt.plot(range(1000), f0_gt, label='GT')
    plt.legend()
plt.show()